# Apply Damage and Save Dataset
Run the function below to apply the damages per image and save the damaged version and their respective masks in their corresponding directory.

In [18]:
import random
import numpy as np
from PIL import Image, ImageDraw, ImageFilter
import os

In [19]:
def apply_and_save_damage_set(original_image_pil, image_identifier,
                              damage_applicator_instance, damage_type_name,
                              base_output_directory):
    """
    Applies a given damage to an image, saves the damaged image and its mask
    into respective subdirectories.

    Args:
        original_image_pil (PIL.Image): The original clean image.
        image_identifier (str or int): A unique identifier for the image (e.g., "Image_001", 
                                     a filename like "hash.jpg", or an index).
        damage_applicator_instance (object): An instance of a damage class (e.g., TearDamage()).
        damage_type_name (str): A string to identify the damage type in filenames (e.g., "Tears").
        base_output_directory (str): Base directory to save the output images.
                                     Subdirectories 'damaged-images' and 'masks' will be created here.
    """
    print(f"Applying {damage_type_name} damage to '{image_identifier}'...")
    
    image_to_damage = original_image_pil.copy() 
    damaged_image, damage_mask = damage_applicator_instance.apply(image_to_damage)

    # Define output subdirectories
    damaged_output_dir = os.path.join(base_output_directory, "damaged-images")
    mask_output_dir = os.path.join(base_output_directory, "masks")

    # Create subdirectories if they don't exist
    os.makedirs(damaged_output_dir, exist_ok=True)
    os.makedirs(mask_output_dir, exist_ok=True)

    # Prepare image identifier for filename (strip extension if it's a filename)
    id_str = str(image_identifier)
    id_str_base, id_str_ext = os.path.splitext(id_str)
    if id_str_ext.lower() in ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp']:
        id_str = id_str_base # Use only the base name without the extension
    
    # Sanitize damage_type_name for filename
    safe_damage_type_name = "".join(c if c.isalnum() else '_' for c in damage_type_name)
        
    base_filename = f"{id_str} {safe_damage_type_name}" 

    damaged_filename = f"{base_filename}.png"
    mask_filename = f"{base_filename}_mask.png"

    damaged_image_path = os.path.join(damaged_output_dir, damaged_filename)
    mask_image_path = os.path.join(mask_output_dir, mask_filename)

    # Save the images
    try:
        damaged_image.save(damaged_image_path)
        damage_mask.save(mask_image_path)
    except Exception as e:
        print(f"  Error saving images for {damage_type_name} on {image_identifier}: {e}")